# GlaSEE pipeline

Full classification pipeline for Sentinel-2 TOA, Sentinel-2 SR, and Landsat 8/9 images. 

## Requirements:
1. Google Earth Engine (GEE) account: used to query imagery and the DEM (if no DEM is provided). Sign up for a free account [here](https://earthengine.google.com/new_signup/). 

2. Google Drive folder: Create a folder where output snow cover statistics will be saved. Enter the name of this folder as the `out_folder` variable below. If you don't create the folder ahead of time, duplicates of the same folder will be created for each output file!

## Define image search settings and paths

In [ ]:
import os
import ee
import geemap
import sys

# -----Define Google Drive folder for exports
# NOTE: Make sure this folder already exists and is the only folder with that name in "My Drive". 
out_folder = 'glacier_snow_cover_exports'

# -----Import pipeline utilities
# Assumes pipeline_utils.py is in the same folder as this notebook
script_path = os.getcwd()
sys.path.append(script_path)
import glasee_pipeline_utils as utils

# -----Define image search settings
# Date and month ranges (inclusive)
date_start = '2014-04-01' 
date_end = '2025-09-15' 
month_start = 4 # April = 4
month_end = 10 # Oct = 10
# Minimum fill portion percentage of the AOI (0–100), used to remove images after mosaicking by day
min_aoi_coverage = 70
# Whether to mask clouds using the respective cloud mask via the geedim package
mask_clouds = True

## Authenticate and/or Initialize Google Earth Engine (GEE)

Replace the project ID with your GEE project. Default = `ee-[GEE-username]`

In [ ]:
project_id = "ee-raineyaberle"

try:
    ee.Initialize(project=project_id)
except:
    ee.Authenticate()
    ee.Initialize(project=project_id)

## Run the pipeline for a single glacier

### Select the Area of Interest (AOI) from the Randolph Glacier Inventory (RGI) dataset

This cell will plot the RGI dataset on a map. To find a glacier, click on the wrench in the upper right toolbox of the map, and use the "Inspector" to click on a polygon and view the its properties. Right click on the "rgi_id" property to highlight and then copy. Replace the `rgi_id` variable below with your selected site. 

In [ ]:
# Load the RGI v7 dataset
rgi = ee.FeatureCollection("projects/ee-raineyaberle/assets/glacier-snow-cover-mapping/RGI2000-v7-G")

# Select a glacier by the RGI v7 ID
rgi_id = 'RGI2000-v7.0-G-01-05740' #RGI2000-v7.0-G-01-11350 (Wolverine), RGI2000-v7.0-G-01-05299 (Gulkana), RGI2000-v7.0-G-01-05740 (Kennicott), 17516 (Casement)
                                 
# Grab the geometry
aoi = rgi.filter(ee.Filter.eq('rgi_id', rgi_id))
aoi = aoi.geometry()
aoi_area = aoi.area().getInfo() # save area [m^2] for splitting date ranges later
print(f"Glacier area = {int(aoi_area/1e6)} km2")

# Create a Map
Map = geemap.Map()
Map.addLayer(rgi, {'color': 'blue', 'opacity':  0.5}, 'RGI v7')
Map.addLayer(aoi, {'color': 'orange', 'opacity': 0.8}, 'AOI')
Map.centerObject(aoi)
Map

### Load the Digital Elevation Model (DEM)

Default: use the ArcticDEM Mosaic where there is > 90 % coverage. Otherwise, use the NASADEM. For sites that use the ArcticDEM Mosaic, elevations are reprojected to the EGM96 geoid to match the vertical datum of NASADEM. 

In [ ]:
# Query GEE for DEM
dem = utils.query_gee_for_dem(aoi)

# Add DEM to map
# grab min and max elevations for color limits
minMax = dem.reduceRegion(reducer=ee.Reducer.minMax(),
                          geometry=aoi, 
                          scale=30,
                          maxPixels=1e9,
                          bestEffort=True)
elev_min = minMax.get('elevation_min')
elev_max = minMax.get('elevation_max')
print(f'Elevation range = {int(elev_min.getInfo())} to {int(elev_max.getInfo())} m')
# colors based on the "terrain" palette from matplotlib
palette = ['#333399', '#0d7fe5', '#00be90','#55dd77','#c6f48e','#e3db8a','#aa926b','#8e6e67','#c6b6b3','#ffffff']
Map.addLayer(dem, {'palette': palette, 'min': elev_min, 'max': elev_max}, 'DEM')

Run Sentinel-2 Top of Atmosphere (TOA): available 2016 onwards

In [ ]:
dataset = "Sentinel-2_TOA"
utils.run_classification_pipeline(aoi, aoi_area, dem, dataset, date_start, date_end, month_start, month_end, 
                                  min_aoi_coverage, mask_clouds, out_folder, rgi_id, scale=None, verbose=False)

Run Sentinel-2 Surface Reflectance (SR): available 2019 onwards

In [ ]:
dataset = "Sentinel-2_SR"
utils.run_classification_pipeline(aoi, aoi_area, dem, dataset, date_start, date_end, month_start, month_end, 
                                  min_aoi_coverage, mask_clouds, out_folder, rgi_id, scale=None, verbose=False)

Run Landsat 8/9 SR: available 2013 onwards

In [ ]:
dataset = "Landsat"
utils.run_classification_pipeline(aoi, aoi_area, dem, dataset, date_start, date_end, month_start, month_end, 
                                  min_aoi_coverage, mask_clouds, out_folder, rgi_id, scale=None, verbose=False)

## Run the pipeline for multiple glaciers

First, create a list of glacier IDs for analysis. 

Below is an example selection of glaciers, where the full RGI v. 7 collection is filtered by RGI O1 region ("o1region") and area ("area_km2"). For the full list of properties available for filtering, see the [RGI v. 7 documentation](https://www.glims.org/rgi_user_guide/products/glacier_product.html#full-list-of-attributes) or run the following command: 

`rgi.first().propertyNames().getInfo()`

In [ ]:
# Load the RGI v. 7 dataset
rgi = ee.FeatureCollection("projects/ee-raineyaberle/assets/glacier-snow-cover-mapping/RGI2000-v7-G")

# Apply filters
rgi_filt = (rgi
            .filter(ee.Filter.eq('o1region', '02')) # Western U.S. & Canada
            .filter(ee.Filter.gte('area_km2', 10)) # area > 10 km2
            .filter(ee.Filter.lte('area_km2', 200)) # area < 200 km2
            )

# Get the list of RGI IDs
id_list = rgi_filt.aggregate_array('rgi_id')
id_list = id_list.getInfo()
print('Number of glaciers selected:', len(id_list))

In [ ]:
# Iterate over RGI IDs
for i in range(0,len(id_list))[0:1]: # using "[0:1]" to only run the first site for testing
    rgi_id = id_list[i]
    print("\nGlacier #", i, '\n---------')
    print("Glacier ID used for output file names:", rgi_id)

    # grab glacier area of interest
    aoi = rgi.filter(ee.Filter.eq('rgi_id', rgi_id))
    aoi = aoi.geometry()
    aoi_area = aoi.area().getInfo() # save area [m^2] for splitting date ranges later
    print(f"Glacier area = {int(aoi_area/1e6)} km2")

    # Query GEE for DEM
    dem = utils.query_gee_for_dem(aoi) 

    # Run pipeline for each dataset
    for dataset in ['Sentinel-2_TOA','Sentinel-2_SR','Landsat']:
        print(print(f'\nRunning {dataset}...'))
        utils.run_classification_pipeline(aoi, aoi_area, dem, dataset, date_start, date_end, month_start, month_end, 
                                          min_aoi_coverage, mask_clouds, out_folder, rgi_id, scale=None, verbose=False)

    

In [ ]:
# Optional: Check job queue statuses
tasks = ee.batch.Task.list()
running_count = len([x for x in tasks if x.state=='RUNNING'])
ready_count = len([x for x in tasks if x.state=='READY'])
completed_count = len([x for x in tasks if x.state=='COMPLETED'])
failed_count = len([x for x in tasks if x.state=='FAILED'])

print('Total number of tasks in queue =', len(tasks))
print('RUNNING tasks =', running_count)
print('READY tasks =', ready_count)
print('COMPLETED tasks =', completed_count)
print('FAILED tasks =', failed_count)